<a href="https://colab.research.google.com/github/ScottTeran/ga_capstone/blob/main/code/text_extraction_google_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install google-cloud-vision

In [ ]:
# pip install google-cloud-storage

In [ ]:
# https://stackoverflow.com/questions/45501082/set-google-application-credentials-in-python-project-to-use-google-api

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/ra-lafferty-13f7704670eb.json"

In [ ]:
import json
import re

from google.cloud import vision
from google.cloud import storage

In [ ]:
def async_detect_documents(gcs_source_uri, gcs_destination_uri):
    
    mime_type = 'application/pdf'
    
    batch_size =  100
    
    client = vision.ImageAnnotatorClient()
    
    feature = vision.Feature(
        type_= vision.Feature.Type.DOCUMENT_TEXT_DETECTION)
    
    gcs_source = vision.GcsSource(uri = gcs_source_uri)
    input_config = vision.InputConfig(
        gcs_source = gcs_source, mime_type = mime_type)
    
    gcs_destination = vision.GcsDestination(uri = gcs_destination_uri)
    output_config = vision.OutputConfig(
        gcs_destination = gcs_destination, 
        batch_size = batch_size)
    
    async_request = vision.AsyncAnnotateFileRequest(
        features = [feature], input_config = input_config,
        output_config = output_config)
    
    operation = client.async_batch_annotate_files(
        requests = [async_request])
    
    print('Waiting for the operation to finish.')
    operation.result(timeout = 400)

In [ ]:
async_detect_documents('gs://ra_lafferty_pdfs/The_Devil_Is_Dead.pdf', 'gs://ra_lafferty_pdfs/txt_files')